<a href="https://colab.research.google.com/github/Chikuji/AzureML/blob/master/2020_09_22_03_TrabalharComDadosAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivos de aprendizado

- Crie e use datastores em um espaço de trabalho do Azure Machine Learning.
-Crie e use conjuntos de dados em um espaço de trabalho do Azure Machine Learning.



Tipos de armazenamento de dados

- Armazenamento do Azure (blob e contêineres de arquivo)
-Armazenamento Azure Data Lake
-Banco de dados SQL do Azure
-Sistema de arquivos do Azure Databricks (DBFS)


# Registrando um armazenamento de dados

Você pode registrá-lo usando a interface gráfica no Azure Machine Learning Studio ou pode usar o SDK do Azure Machine Learning

In [ ]:
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

# Register a new datastore
blob_ds = Datastore.register_azure_blob_container(workspace=ws,
    datastore_name='blob_data',
    container_name='data_container',
    account_name='az_store_acct',
    account_key='123456abcde789…')

Gerenciando datastores

In [ ]:
# lista os nomes de cada armazenamento de dados no espaço de trabalho.
for ds_name in ws.datastores:
    print(ds_name)

In [ ]:
# Você pode obter uma referência para qualquer armazenamento de dados usando o método Datastore.get ()
blob_store = Datastore.get(ws, datastore_name='blob_data')

In [ ]:
# Você pode recuperar usando o método get_default_datastore () de um objeto Workspace
default_store = ws.get_default_datastore()

In [ ]:
# Para alterar o armazenamento de dados padrão, use o método set_default_datastore ()
ws.set_default_datastore('blob_data')

# Trabalhando diretamente com um armazenamento de dados

In [ ]:
blob_ds.upload(src_dir='/files',
               target_path='/data/files',
               overwrite=True, show_progress=True)

blob_ds.download(target_path='downloads',
                 prefix='/data',
                 show_progress=True)

Usando referências de dados

- Download : o conteúdo do caminho associado à referência de dados é baixado para o contexto de computação onde o experimento está sendo executado.
-Upload : os arquivos gerados por seu script de experimento são enviados para o armazenamento de dados após a conclusão da execução.
-Montagem : o caminho no armazenamento de dados é montado como armazenamento remoto no contexto de computação do experimento, permitindo que o conteúdo seja acessado remotamente (observe que este modo está disponível apenas quando o experimento é executado em um destino de computação remoto - você não pode usar este modo com computação local).


Para passar a referência de dados para um script de experimento, você deve definir um parâmetro de script conforme mostrado aqui:

In [ ]:
data_ref = blob_ds.path('data/files').as_download(path_on_compute='training_data')
estimator = SKLearn(source_directory='experiment_folder',
                    entry_script='training_script.py'
                    compute_target='local',
                    script_params = {'--data_folder': data_ref})



Em seu script de treinamento, você pode recuperar o parâmetro e usá-lo como uma pasta local:

In [ ]:
import os
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--data_folder', type=str, dest='data_folder')
args = parser.parse_args()
data_files = os.listdir(args.data_folder)

# Introdução aos conjuntos de dados

Criação e registro de conjuntos de dados tabulares

In [ ]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
csv_paths = [(blob_ds, 'data/files/current_data.csv'),
             (blob_ds, 'data/files/archive/*.csv')]
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

Criação e registro de conjuntos de dados de arquivo

In [ ]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
file_ds = Dataset.File.from_files(path=(blob_ds, 'data/files/images/*.jpg'))
file_ds = file_ds.register(workspace=ws, name='img_files')

Recuperando um conjunto de dados registrado

In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Get a dataset from the workspace datasets collection
ds1 = ws.datasets['csv_table']

# Get a dataset by name from the datasets class
ds2 = Dataset.get_by_name(ws, 'img_files')

Controle de versão do conjunto de dados

In [ ]:
img_paths = [(blob_ds, 'data/files/images/*.jpg'),
             (blob_ds, 'data/files/images/*.png')]
file_ds = Dataset.File.from_files(path=img_paths)
file_ds = file_ds.register(workspace=ws, name='img_files', create_new_version=True)

Recuperando uma versão específica do conjunto de dados

In [ ]:
img_ds = Dataset.get_by_name(workspace=ws, name='img_files', version=2)

# Use conjuntos de dados

Trabalhar com um conjunto de dados diretamente

In [ ]:
df = tab_ds.to_pandas_dataframe()
# code to work with dataframe goes here

In [ ]:
for file_path in file_ds.to_path():
    print(file_path)
# método to_path () para retornar uma lista dos caminhos de arquivo encapsulados pelo conjunto de dados


Passar um conjunto de dados para um script de experimento

In [ ]:
estimator = SKLearn( source_directory='experiment_folder',
                     entry_script='training_script.py',
                     compute_target='local',
                     inputs=[tab_ds.as_named_input('csv_data')],
                     pip_packages=['azureml-dataprep[pandas]')

Como o script precisará trabalhar com um objeto Dataset , você deve incluir o pacote azureml-sdk completo ou o pacote azureml-dataprep com a biblioteca extra pandas no ambiente de computação do script.

No próprio script do experimento, você pode acessar a entrada e trabalhar com o objeto Dataset ao qual ele faz referência, assim:

In [ ]:
run = Run.get_context()
data = run.input_datasets['csv_data'].to_pandas_dataframe()

Ao passar um conjunto de dados de arquivo, você deve especificar o modo de acesso. Por exemplo:

In [ ]:
estimator = Estimator( source_directory='experiment_folder',
                     entry_script='training_script.py'
                     compute_target='local',
                     inputs=[img_ds.as_named_input('img_data').as_download(path_on_compute='data')],
                     pip_packages=['azureml-dataprep[pandas]')